In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
### Load the trained model, scaler pickle, onehot
model = load_model("artifacts/model/model.h5")

## Load the encoder scaler
with open("artifacts/preprocessing/OHE_geo.pkl", "rb") as file:
    onehot_encoder_geo = pickle.load(file)

with open("artifacts/preprocessing/OHE_gender.pkl", "rb") as file:
    onehot_encoder_gender = pickle.load(file)

with open("artifacts/preprocessing/scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
'''input_data = {'CreditScore': 502,
 'Geography': 'France',
 'Gender': 'Female',
 'Age': 42,
 'Tenure': 8,
 'Balance': 159660.8,
 'NumOfProducts': 3,
 'HasCrCard': 1,
 'IsActiveMember': 0,
 'EstimatedSalary': 113931.57}'''

"input_data = {'CreditScore': 502,\n 'Geography': 'France',\n 'Gender': 'Female',\n 'Age': 42,\n 'Tenure': 8,\n 'Balance': 159660.8,\n 'NumOfProducts': 3,\n 'HasCrCard': 1,\n 'IsActiveMember': 0,\n 'EstimatedSalary': 113931.57}"

In [5]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Prima\Github\Churn_Modeling_Classification_ANN_DLProject\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
# One-hot encode 'Gender'
gender_encoded = onehot_encoder_gender.transform([[input_data['Gender']]])
gender_encoded_df = pd.DataFrame(gender_encoded, columns=onehot_encoder_gender.get_feature_names_out(['Gender']))
gender_encoded_df

c:\Prima\Github\Churn_Modeling_Classification_ANN_DLProject\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Gender_Female,Gender_Male
0,0.0,1.0


In [8]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop(["Geography", "Gender"],axis=1),gender_encoded_df, geo_encoded_df],axis=1)
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,600,40,3,60000,2,1,1,50000,0.0,1.0,1.0,0.0,0.0


In [9]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.10479359, -0.69539349, -0.25781119,  0.80843615,
         0.64920267,  0.97481699, -0.87683221, -0.91324755,  0.91324755,
         1.00150113, -0.57946723, -0.57638802]])

In [10]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 456ms/step


array([[0.05260158]], dtype=float32)

In [11]:
prediction_proba = prediction[0][0]

In [12]:
prediction_proba

0.052601583

In [13]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
